In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm_notebook as tqdm
from collections import defaultdict

In [2]:
path = Path('data/')

## Read in data

In [3]:
print('Reading in train.csv')
train_data = pd.read_csv(path/'train.csv')
print('Reading in train_labels.csv')
train_labels = pd.read_csv(path/'train_labels.csv')
print('Reading in test.csv')
test_data = pd.read_csv(path/'test.csv')
print('Reading in specs.csv')
specs = pd.read_csv(path/'specs.csv')

Reading in train.csv
Reading in train_labels.csv
Reading in test.csv
Reading in specs.csv


## Reduce the size of the data frames

In [4]:
## Function to reduce the DF size
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [5]:
train_data = reduce_mem_usage(train_data)
train_labels = reduce_mem_usage(train_labels)
test_data = reduce_mem_usage(test_data)
specs = reduce_mem_usage(specs)

Mem. usage decreased to 778.73 Mb (18.2% reduction)
Mem. usage decreased to  0.49 Mb (48.2% reduction)
Mem. usage decreased to 79.40 Mb (18.2% reduction)
Mem. usage decreased to  0.01 Mb (0.0% reduction)


In [6]:
# helper function to decompose dates into more relevant parts
def explode_date(col):
    dayofyear = col.dt.dayofyear
    weekofyear = col.dt.weekofyear
    weekday = col.dt.weekday
    month = col.dt.month
    year = col.dt.year
    hour = col.dt.hour
    quarter = col.dt.quarter
    return pd.DataFrame({'dayofyear':dayofyear,
                         'weekofyear':weekofyear,
                         'weekday':weekday,
                         'quarter':quarter,
                         'month':month,
                         'year':year,
                         'hour':hour})

Main funtion that aggregates the sequence data

In [7]:
def reduce_data(install_id,data,is_train_data=True):
    rows = []
    for idx,df in tqdm(data.groupby('installation_id')):
        rows.extend(reduce_data_per_user(df))
    data = pd.DataFrame(rows)
    if is_train_data:
        return train_labels[['game_session','accuracy_group']]\
                        .merge(data,on='game_session',how='inner')\
                        .drop(['game_session'],axis=1)
    else:
        return data.groupby('installation_id',as_index=False).apply(lambda x: x.iloc[-1]).drop(['game_session'],axis=1)

In [8]:
specs.head()

,event_id,info,args
0,2b9272f4,The end of system-initiated feedback (Correct)...,"[{""name"":""game_time"",""type"":""int"",""info"":""mill..."
1,df4fe8b6,The end of system-initiated feedback (Incorrec...,"[{""name"":""game_time"",""type"":""int"",""info"":""mill..."
2,3babcb9b,The end of system-initiated instruction event ...,"[{""name"":""game_time"",""type"":""int"",""info"":""mill..."
3,7f0836bf,The end of system-initiated instruction event ...,"[{""name"":""game_time"",""type"":""int"",""info"":""mill..."
4,ab3136ba,The end of system-initiated instruction event ...,"[{""name"":""game_time"",""type"":""int"",""info"":""mill..."


In [9]:
event_description_dict = {row.event_id:row.info for idx,row in specs.iterrows()}

In [10]:
unique_titles = set(train_data.title.unique()).union(set(test_data.title.unique()))

In [11]:
# reduces data with respect to each installation id
def reduce_data_per_user(user_dataframe):
    event_counts = {'total_events':0,'clips_count':0,'assessment_count':0,'activity_count':0,'game_count':0,
                    'correct_events_count':0,'incorrect_events_count':0}
    specific_event_counts = dict(zip(specs.event_id,[0 for _ in range(specs.shape[0])]))
    unique_titles_counts = dict(zip(unique_titles,[0 for _ in range(len(unique_titles))]))
    median_session_times = []
    df_features = []
    for idx,session in user_dataframe.groupby('game_session'):
        if session.type.iloc[0]=='Assessment':
            assessment_info = {'assess_world':session.world.iloc[0],
                               'assess_title':session.title.iloc[0],
                               'game_session':session.game_session.iloc[0],
                               'installation_id':session.installation_id.iloc[0]
                              }
            
            df_features.append({**event_counts,**assessment_info,**specific_event_counts,
                                **unique_titles_counts})
            event_counts['assessment_count'] += (session.type == 'Assessment').sum()
            unique_titles_counts[session.iloc[0].title]+=1
        else:
            event_counts['total_events'] += session.event_id.count()
            event_counts['clips_count'] += (session.type == 'Clip').sum()
            event_counts['activity_count'] += (session.type == 'Activity').sum()
            event_counts['game_count'] += (session.type == 'Game').sum()
            event_counts['correct_events_count'] += (session.event_id
                                                     .map(event_description_dict)
                                                     .str.contains('Correct').sum())
            event_counts['incorrect_events_count'] += (session.event_id
                                                       .map(event_description_dict)
                                                       .str.contains('Incorrect').sum())
            specific_event_counts.update(dict(session.event_id.value_counts()))
            unique_titles_counts[session.iloc[0].title]+=1
    return df_features

In [12]:
train_reduced = reduce_data(train_labels.installation_id.unique(),train_data)
test_reduced = reduce_data(test_data.installation_id.unique(),test_data, is_train_data = False)

In [13]:
train_reduced.to_csv(path/'train_reduced.csv')
test_reduced.to_csv(path/'test_reduced.csv')

In [29]:
train_reduced = pd.read_csv(path/'train_reduced.csv',index_col=0)
test_reduced = pd.read_csv(path/'test_reduced.csv',index_col=0)

# Modeling

In [37]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from sklearn.compose import ColumnTransformer
from imblearn.over_sampling import SMOTENC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier
from sklearn.metrics import cohen_kappa_score,confusion_matrix
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer
import xgboost as xgb
from rfpimp import *
from plotnine import *

In [38]:
worlds = list(set(train_reduced.assess_world.unique()).union(test_reduced.assess_world.unique()))
titles = list(set(train_reduced.assess_title.unique()).union(test_reduced.assess_title.unique()))

In [32]:
# train,val = train_test_split(train_reduced,test_size=.2)
# X_col_names = [col for col in train_reduced.columns if col != 'accuracy_group']
# train_x = train.loc[:,X_col_names]
# train_y = train.loc[:,'accuracy_group']
# val_x = val.loc[:,X_col_names]
# val_y = val.loc[:,'accuracy_group']

In [33]:
X_col_names = [col for col in train_reduced.columns if col not in ['accuracy_group','installation_id']]
train_x = train_reduced.loc[:,X_col_names]
train_y = train_reduced.loc[:,'accuracy_group']

In [34]:
smote = SMOTENC([7,8])
train_x_smote,train_y_smote = smote.fit_resample(train_x,train_y)

In [35]:
train_x_smote = pd.DataFrame(train_x_smote,columns=X_col_names)
train_x_smote['installation_id'] = 1
train_x['installation_id'] = 1

In [36]:
def evaluate_model(model,x,y):
    cv_scores = cross_val_score(model,x,y,scoring=make_scorer(cohen_kappa_score,weights='quadratic'),cv=5,n_jobs=5)
    cv_mean_score = cv_scores.mean()
    cv_sd = cv_scores.std()
    train_score = cohen_kappa_score(model.fit(x,y).predict(x),y,weights='quadratic')
    print(f'Train score:{train_score}')
    print(f'Cross Validation Mean: {cv_mean_score:.3f} SD: {cv_sd:.3f}')

In [46]:
preprocessor = ColumnTransformer([('onehot1',OneHotEncoder(),['assess_world']),
                                  ('onehot2',OneHotEncoder(),['assess_title']),
                                  ('drop', 'drop',['installation_id'])],
                                  remainder='passthrough')

In [47]:
rf_pipe = Pipeline([('col_transform',preprocessor),
                 ('cls',RandomForestClassifier(n_estimators=200,
                                               max_features = int(np.sqrt(train_x.shape[1])),
                                               min_samples_leaf=15,
                                               max_depth=None,
                                               n_jobs=8))])

In [48]:
print(f'Random Forest SMOTE results')
#evaluate_model(rf_pipe,train_x_smote,train_y_smote)
rf_pipe.fit(train_x_smote,train_y_smote)

Random Forest SMOTE results


Pipeline(memory=None,
         steps=[('col_transform',
                 ColumnTransformer(n_jobs=None, remainder='passthrough',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('onehot1',
                                                  OneHotEncoder(categorical_features=None,
                                                                categories=None,
                                                                drop=None,
                                                                dtype=<class 'numpy.float64'>,
                                                                handle_unknown='error',
                                                                n_values=None,
                                                                sparse=True),
                                                  ['assess_world']),
                                    

In [26]:
def prepare_submission(model,test_data):
    install_id = test_data.installation_id.to_numpy()
    test_predictions = model.predict(test_data.drop(['installation_id'],axis=1))
    submission_df = pd.DataFrame({'installation_id':install_id,
                                  'accuracy_group':test_predictions})
    return submission_df
test = prepare_submission(rf_pipe,test_reduced)

In [27]:
test.to_csv('submission.csv',index=False)